In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2033, 0.1994, 0.2010])
])
testData = datasets.CIFAR10(
    root = "./data",
    train = False,
    download = True,
    transform = transform
)
testLoader = torch.utils.data.DataLoader(
    testData, batch_size = 64, shuffle = True
)
dataset = datasets.CIFAR10(
    root = "./data",
    train = True,
    download = True,
    transform = transform
)
# Chia chỉ số cho tập huấn luyện và tập kiểm định
numTrain = len(dataset)
indices = list(range(numTrain))
split = int(0.9 * numTrain)
# Trộn ngẫu nhiên các chỉ số
torch.manual_seed(42)
np.random.shuffle(indices)
trainIdx, validIdx = indices[:split], indices[split:]
trainSampler = SubsetRandomSampler(trainIdx)
validSampler = SubsetRandomSampler(validIdx)
# Tạo DataLoader
trainLoader = torch.utils.data.DataLoader(dataset, batch_size = 64, sampler = trainSampler)
validLoader = torch.utils.data.DataLoader(dataset, batch_size = 64, sampler = validSampler)
print(len(trainLoader), len(validLoader))

Files already downloaded and verified
Files already downloaded and verified
704 79


In [3]:
class VGG16(nn.Module):
  def __init__(self, numClasses = 10):
    super(VGG16, self).__init__()
    self.features = nn.Sequential(
        # block 1
        nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 2
        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 3
        nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 4
        nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 5
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7 * 7 * 512, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, numClasses)
    )
  def forward(self, out):
    out = self.features(out)
    out = out.reshape(out.size(0), -1)
    out = self.classifier(out)
    return out


In [4]:
numClasses = 10
numEpochs = 20
learningRate = 0.001

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16(numClasses).to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learningRate)
totalStep = len(trainLoader)

In [7]:
for epoch in range(numEpochs):
    progress_bar = tqdm(enumerate(trainLoader), total=len(trainLoader), desc=f'Epoch {epoch + 1}/{numEpochs}')  
    for i, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loses = criterion(outputs, labels)
        optimizer.zero_grad()
        loses.backward()
        optimizer.step()
        progress_bar.set_postfix(loss = loses.item())
    torch.save(model.state_dict(), f'/kaggle/working/model_epoch_{epoch + 1}.pth')
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(validLoader):
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        print("Accuracy of the model on validation images:", (correct *100.0 / total))

Epoch 1/20: 100%|██████████| 704/704 [13:07<00:00,  1.12s/it, loss=2.15]


Accuracy of the model on validation images: 27.46


Epoch 2/20: 100%|██████████| 704/704 [13:07<00:00,  1.12s/it, loss=2.45]


Accuracy of the model on validation images: 34.08


Epoch 3/20: 100%|██████████| 704/704 [13:08<00:00,  1.12s/it, loss=1.09]


...

Accuracy of the model on validation images: 78.5


Epoch 19/20: 100%|██████████| 704/704 [13:14<00:00,  1.13s/it, loss=1.57]  


Accuracy of the model on validation images: 80.7


Epoch 20/20: 100%|██████████| 704/704 [13:15<00:00,  1.13s/it, loss=0.975] 


Accuracy of the model on validation images: 82.1


In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    model.eval()
    for images, labels in testLoader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    print("Accuracy of the model on test images:", (correct / total) * 100)

Accuracy of the model on test images: 84.73
